In [ ]:
import numpy as np 
import pandas as pd
import os
import gc
from sklearn.preprocessing import MinMaxScaler
from keras import losses
from keras.layers import Input, Dense, Dropout
from keras.models import Model, Sequential
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam
print(os.listdir("../input"))
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold, KFold
from collections import Counter, defaultdict
import lightgbm
import xgboost
from sklearn import metrics
from sklearn.preprocessing import Imputer
from sklearn import metrics
import gc
import os

In [ ]:
dtypes = {
    'MachineIdentifier':              'category',
    'ProductName':                        'int8',
    'EngineVersion':                      'int8',
    'AppVersion':                         'int8',
    'AvSigVersion':                      'int16',
    'IsBeta':                             'int8',
    'RtpStateBitfield':                   'int8',
    'IsSxsPassiveMode':                   'int8',
    'DefaultBrowsersIdentifier':         'int16',
    'AVProductStatesIdentifier':         'int16',
    'AVProductsInstalled':                'int8',
    'AVProductsEnabled':                  'int8',
    'HasTpm':                             'int8',
    'CountryIdentifier':                 'int16',
    'CityIdentifier':                  'float32',
    'OrganizationIdentifier':             'int8',
    'GeoNameIdentifier':                 'int16',
    'LocaleEnglishNameIdentifier':       'int16',
    'Platform':                           'int8',
    'Processor':                          'int8',
    'OsVer':                                         'int8',
    'OsBuild':                                       'int8',
    'OsSuite':                                       'int8',
    'OsPlatformSubRelease':                          'int8',
    'OsBuildLab':                                   'int16',
    'SkuEdition':                                    'int8',
    'IsProtected':                                'float16',
    'AutoSampleOptIn':                               'int8',
    'PuaMode':                                   'category',
    'SMode':                                      'float16',
    'IeVerIdentifier':                              'int16',
    'SmartScreen':                                   'int8',
    'Firewall':                                   'float16',
    'UacLuaenable':                                  'int8',
    'Census_MDC2FormFactor':                         'int8',
    'Census_DeviceFamily':                           'int8',
    'Census_OEMNameIdentifier':                     'int16',
    'Census_OEMModelIdentifier':                  'float32',
    'Census_ProcessorCoreCount':                  'float16',
    'Census_ProcessorManufacturerIdentifier':        'int8',
    'Census_ProcessorModelIdentifier':                        'int16',
    'Census_ProcessorClass':                                   'int8',
    'Census_PrimaryDiskTotalCapacity':                      'float32',
    'Census_PrimaryDiskTypeName':                              'int8',
    'Census_SystemVolumeTotalCapacity':                     'float32',
    'Census_HasOpticalDiskDrive':                              'int8',
    'Census_TotalPhysicalRAM':                              'float32',
    'Census_ChassisTypeName':                                  'int8',
    'Census_InternalPrimaryDiagonalDisplaySizeInInches':    'float16',
    'Census_InternalPrimaryDisplayResolutionHorizontal':    'float16',
    'Census_InternalPrimaryDisplayResolutionVertical':      'float16',
    'Census_PowerPlatformRoleName':                            'int8',
    'Census_InternalBatteryType':                              'int8',
    'Census_InternalBatteryNumberOfCharges':                'float32',
    'Census_OSVersion':                                       'int16',
    'Census_OSArchitecture':                                   'int8',
    'Census_OSBranch':                                         'int8',
    'Census_OSBuildNumber':                                   'int16',
    'Census_OSBuildRevision':                                 'int16',
    'Census_OSEdition':                                        'int8',
    'Census_OSSkuName':                              'int8',
    'Census_OSInstallTypeName':                      'int8',
    'Census_OSInstallLanguageIdentifier':            'int8',
    'Census_OSUILocaleIdentifier':                  'int16',
    'Census_OSWUAutoUpdateOptionsName':              'int8',
    'Census_IsPortableOperatingSystem':              'int8',
    'Census_GenuineStateName':                       'int8',
    'Census_ActivationChannel':                      'int8',
    'Census_IsFlightingInternal':                 'float16',
    'Census_IsFlightsDisabled':                   'float16',
    'Census_FlightRing':                             'int8',
    'Census_ThresholdOptIn':                      'float16',
    'Census_FirmwareManufacturerIdentifier':        'int16',
    'Census_FirmwareVersionIdentifier':           'float16',
    'Census_IsSecureBootEnabled':                    'int8',
    'Census_IsWIMBootEnabled':                    'float16',
    'Census_IsVirtualDevice':                     'float16',
    'Census_IsTouchEnabled':                         'int8',
    'Census_IsPenCapable':                           'int8',
    'Census_IsAlwaysOnAlwaysConnectedCapable':    'float16',
    'Wdft_IsGamer':                               'float16',
    'Wdft_RegionIdentifier':                         'int8',
    'HasDetections':                                 'int8' }


In [ ]:
train_df = pd.read_csv('../input/microsoft-shrink/new_train.csv', dtype = dtypes)
test_df = pd.read_csv('../input/microsoft-shrink/new_test.csv', dtype = dtypes)

In [ ]:
gc.collect()

In [ ]:
#test_df['OsSuite'].value_counts()

In [ ]:
nulls_cols = ['PuaMode', 'Census_IsFlightingInternal','Census_ThresholdOptIn', 'Census_IsWIMBootEnabled', 'Unnamed: 0']
for col in nulls_cols:
    del train_df[col]
    del test_df[col]
gc.collect()

In [ ]:
gc.collect()

In [ ]:
train_df.tail()

In [ ]:
train_df.shape, test_df.shape

In [ ]:
gc.collect()

In [ ]:
full_cat_cols = ['ProductName','EngineVersion', 'AppVersion', 'AvSigVersion', 'IsBeta', 'RtpStateBitfield', 'IsSxsPassiveMode', 'DefaultBrowsersIdentifier',
                'AVProductStatesIdentifier', 'AVProductsInstalled', 'AVProductsEnabled', 'HasTpm', 'CountryIdentifier', 'CityIdentifier',
                'OrganizationIdentifier', 'GeoNameIdentifier', 'LocaleEnglishNameIdentifier', 'Platform', 'Processor', 'OsVer' , 'OsBuild',
                'OsSuite', 'OsPlatformSubRelease', 'OsBuildLab', 'SkuEdition', 'IsProtected', 'AutoSampleOptIn', 'SMode', 'IeVerIdentifier', 'SmartScreen', 'Firewall', 'UacLuaenable',
                'Census_MDC2FormFactor', 'Census_DeviceFamily', 'Census_OEMNameIdentifier', 'Census_OEMModelIdentifier', 'Census_ProcessorCoreCount',
                'Census_ProcessorManufacturerIdentifier', 'Census_ProcessorModelIdentifier', 'Census_ProcessorClass', 'Census_PrimaryDiskTypeName',
                'Census_HasOpticalDiskDrive','Census_TotalPhysicalRAM',  'Census_ChassisTypeName',  'Census_InternalPrimaryDiagonalDisplaySizeInInches', 'Census_InternalPrimaryDisplayResolutionHorizontal',
                'Census_PowerPlatformRoleName', 'Census_InternalBatteryType', 'Census_InternalBatteryNumberOfCharges', 'Census_OSVersion', 'Census_OSArchitecture', 'Census_OSBranch',
                'Census_OSBuildNumber', 'Census_OSEdition', 'Census_OSSkuName', 'Census_OSInstallTypeName', 'Census_OSUILocaleIdentifier', 'Census_OSWUAutoUpdateOptionsName',
                 'Census_GenuineStateName', 'Census_ActivationChannel', 'Census_IsFlightsDisabled', 'Census_FirmwareManufacturerIdentifier', 'Census_IsSecureBootEnabled',
                 'Census_IsVirtualDevice', 'Census_IsTouchEnabled','Census_IsPenCapable', 'Census_IsAlwaysOnAlwaysConnectedCapable', 'Wdft_RegionIdentifier',
                'Census_InternalPrimaryDisplayResolutionVertical', 'Wdft_IsGamer']

In [ ]:
for col in full_cat_cols:
    train_df[col] = train_df[col].fillna(train_df[col].mode().iloc[0]).astype(dtypes[col])
    test_df[col] = test_df[col].fillna(test_df[col].mode().iloc[0]).astype(dtypes[col])
    print(col)

In [ ]:
gc.collect()

In [ ]:
num_cols = ['Census_PrimaryDiskTotalCapacity', 'Census_SystemVolumeTotalCapacity', 'Census_OSBuildRevision', 'Census_FirmwareVersionIdentifier' ]
for col in num_cols:
    train_df[col].fillna((train_df[col].mean()), inplace=True)
    test_df[col].fillna((test_df[col].mean()), inplace=True)
    train_df[col] = train_df[col].astype(dtypes[col])
    test_df[col] = test_df[col].astype(dtypes[col])
    print(col)

In [ ]:
gc.collect()

In [ ]:
def DefaultBrowsersIdentifier_encoding(x):
    if x == -1: return 0
    elif x== 2: return 2
    elif x == 12: return 12
    elif x == 8: return 8
    elif x == 3: return 3
    elif x == 1: return 1
    elif x == 21: return 21
    elif x == 11: return 11
    else : return 999    
    
def AvSigVersion_encoding(x):
    if x <= 136: return 0
    elif 136 < x <=  371 : return 1
    elif 371 < x <= 763 : return 2
    else : return 4
    
def AppVersion_encoding(x):
    if x == 0 : return 0
    elif 1 <= x <  5 : return 2
    elif 5 <= x <= 10 : return 3
    else : return 4
    
def EngineVersion_encoding(x):
    if x == 0 : return 0
    elif x ==2: return 2
    else : return 1
    
def AVProductStatesIdentifier_encoding(x):
    if x == 0 : return 0
    elif x ==5: return 5
    elif x ==3: return 3
    elif x > 10: return 4
    else : return 1
    
def AVProductsInstalled_encoding(x):
    if x == 0 : return 0
    elif x ==1: return 1
    elif x ==2: return 2
    else : return 3

def AVProductsEnabled_encoding(x):
    if x == 0 : return 0
    elif x ==1: return 1
    else : return 2

def CountryIdentifier_encoding(x):
    if 0 <= x < 50 : return 0
    elif 50 <= x < 100: return 1
    else : return 2
    
def CityIdentifier_encoding(x):
    if x <= 10 : return 0
    elif 10 < x <= 50: return 1
    else : return 2
    
def OrganizationIdentifier_encoding(x):
    if x == 0 : return 0
    elif x == 1 : return 1
    elif x == -1 : return 2
    else : return 3
    
def GeoNameIdentifier_encoding(x):
    if x <= 10 : return 0
    elif 10 < x <= 20 : return 1
    elif 20 < x <= 50 : return 2
    else : return 3
    
def LocaleEnglishNameIdentifier_encoding(x):
    if x <= 5 : return 0
    elif 5 < x <= 11 : return 1
    elif 11 < x <= 32 : return 2
    else : return 3
    
def Platform_encoding(x):
    if x == 0 : return 0
    else : return 1

def Processor_encoding(x):
    if x == 0 : return 0
    else : return 1
    
def OsVer_encoding(x):
    if x == 1 : return 1
    elif x == 2 : return 2
    else : return 0

def OsBuild_encoding(x):
    if 0 <= x <=  7: return x
    else : return 8
    
def OsSuite_encoding(x):
    if 0 <= x <=  1: return x
    else : return 2
    
def OsPlatformSubRelease_encoding(x):
    if x ==  0: return 0
    elif x == 2: return 1
    else : return 2
    
def OsBuildLab_encoding(x):
    if x ==  0: return 0
    elif x == 2: return 1
    elif x == 5: return 2
    elif x == 11: return 3
    elif x == 8: return 4
    elif x == 9: return 5
    elif x == 16: return 6
    elif x == 25: return 7
    elif x == 24: return 8
    else : return 9

def SkuEdition_encoding(x):
    if x ==  0: return 0
    elif x == 1: return 1
    else : return 2
    
def IeVerIdentifier_encoding(x):
    if x <=  0: return 0
    elif 0 < x <= 3 : return 1
    elif 3 < x <= 7: return 2
    else : return 3
    
def SmartScreen_encoding(x):
    if x ==  0: return 0
    elif x == -1 : return 1
    elif x == 1 : return 2
    elif x == 2 : return 3
    elif x == 3 : return 4
    else : return 5
    
def UacLuaenable_encoding(x):
    if x ==  0: return 0
    elif x == 1 : return 1
    else : return 2
    
def Census_MDC2FormFactor_encoding(x):
    if 0 <= x <= 5: return x
    else : return 6
    
def Census_DeviceFamily_encoding(x):
    if x == 1: return 1
    else : return 0
    
def Census_OEMNameIdentifier_encoding(x):
    if x <= 2 : return 0
    elif 2 < x <= 7 :return 1
    elif 7 < x <= 10 :return 2
    else : return 3
    
def Census_OEMModelIdentifier_encoding(x):
    if x <= 59 : return 0
    elif 59 < x <= 439 :return 1
    elif 439 < x <= 2156 :return 2
    else : return 3

def Census_ProcessorCoreCount_encoding(x):
    if x == 4.0 : return 0
    elif x == 2.0 : return 1
    elif x == 8.0 : return 2
    else : return 3
    
def Census_ProcessorManufacturerIdentifier_encoding(x):
    if x == -1 : return 2
    elif x == 1 : return 1
    else : return 0
    
def Census_ProcessorModelIdentifier_encoding(x):
    if x <= 44 : return 0
    elif 44 < x <= 115 : return 1
    elif 115 < x <= 223 : return 2
    else : return 3

def Census_ProcessorClass_encoding(x):
    if x == -1 : return 0
    else : return 1
    
def Census_PrimaryDiskTypeName_encoding(x):
    if 0<= x <= 3 : return x
    else : return 4

def Census_TotalPhysicalRAM_encoding(x):
    if x <= 2050 : return 0
    elif 2050 < x <= 4100: return 1
    elif 4100 < x <= 8200: return 2
    else : return 3

def Census_ChassisTypeName_encoding(x):
    if x ==0 : return 0
    elif x == 1: return 1
    else : return 2
    
def Census_InternalPrimaryDiagonalDisplaySizeInInches_encoding(x):
    if x < 10 : return 0
    elif 10 <= x < 14: return 1
    elif 14<= x <= 17: return 2
    else : return 3

def Census_InternalPrimaryDisplayResolutionHorizontal_encoding(x):
    if x < 1300 : return 0
    elif 1300 <= x < 1800: return 1
    elif 1800<= x <= 2000: return 2
    else : return 3

def Census_InternalPrimaryDisplayResolutionVertical_encoding(x):
    if x < 770 : return 0
    elif 770 <= x < 950: return 1
    else : return 2
    
def Census_PowerPlatformRoleName_encoding(x):
    if 0 <= x <= 3 : return x
    else : return 4

def Census_InternalBatteryType_encoding(x):
    if x == 7 : return 1
    elif x == 9: return 1
    elif x == 14: return 1
    elif x == 25: return 2
    elif x == 5: return 2
    elif x == -1: return 3
    else : return 4

def Census_InternalBatteryNumberOfCharges_encoding(x):
    if x < 1 : return 0
    elif x > 4000: return 1
    else : return 2
    
def Census_OSVersion_encoding(x):
    if x <= 2 : return 0
    elif 2< x <= 14: return 1
    elif 14 < x <= 37: return 2
    else : return 3
    
def Census_OSBranch_encoding(x):
    if x <= 1 : return 0
    elif 1< x <= 3: return 1
    elif 3 < x <= 6: return 2
    else : return 3
    
def Census_OSBuildNumber_encoding(x):
    if x == 0 : return 0
    elif x == 2: return 1
    elif x == 5: return 2
    elif x == 1: return 3
    elif x == 3: return 4
    elif x == 4: return 5
    else : return 6

def Census_OSEdition_encoding(x):
    if x == 0 : return 0
    elif x == 1: return 1
    elif x == 2: return 2
    else : return 3

def Census_OSSkuName_encoding(x):
    if x == 0 : return 0
    elif x == 1: return 1
    elif x == 2: return 2
    else : return 3

def Census_OSInstallLanguageIdentifier_encoding(x):
    if x == 1 : return 0
    elif x ==13 : return 1 
    elif x == 8: return 2
    elif x == 2: return 3
    elif x == 9: return 4
    elif x == 5: return 5
    elif x == 0: return 6
    elif x == 11: return 7
    elif x == 18: return 8
    elif x == 10: return 9
    elif x == 12: return 10
    elif x == 4: return 11
    elif x == 14: return 12
    elif x == 19: return 13
    elif x == 15: return 14
    else : return 15

def Census_OSUILocaleIdentifier_encoding(x):
    if x == 1 : return 0
    elif x ==13 : return 1 
    elif x == 8: return 2
    elif x == 2: return 3
    elif x == 9: return 4
    elif x == 5: return 5
    elif x == 0: return 6
    elif x == 11: return 7
    elif x == 19: return 8
    elif x == 10: return 9
    elif x == 12: return 10
    elif x == 4: return 11
    elif x == 14: return 12
    elif x == 20: return 13
    elif x == 15: return 14
    else : return 15 

def Census_ActivationChannel_encoding(x):
    if 0 <= x <= 3 : return x
    else : return 4

def Census_FirmwareManufacturerIdentifier_encoding(x):
    if x == 1 : return 0
    elif x ==0 : return 1 
    elif x == 14: return 2
    elif x == 2: return 3
    elif x == 4: return 4
    elif x == 6: return 5
    elif x == 3: return 6
    elif x == -1: return 7
    elif x == 24: return 8
    elif x == 20: return 9
    elif x == 19: return 10
    else : return 11

In [ ]:
for df in [train_df, test_df]:
    df['DefaultBrowsersIdentifier'] = df['DefaultBrowsersIdentifier'].apply(lambda x : DefaultBrowsersIdentifier_encoding(x) ).astype(dtypes['DefaultBrowsersIdentifier'])
    df['AvSigVersion'] = df['AvSigVersion'].apply(lambda x : AvSigVersion_encoding(x) ).astype(dtypes['AvSigVersion'])
    df['AppVersion'] = df['AppVersion'].apply(lambda x : AppVersion_encoding(x) ).astype(dtypes['AppVersion'])
    df['EngineVersion'] = df['EngineVersion'].apply(lambda x : EngineVersion_encoding(x) ).astype(dtypes['EngineVersion'])
    df['AVProductStatesIdentifier'] = df['AVProductStatesIdentifier'].apply(lambda x : AVProductStatesIdentifier_encoding(x) ).astype(dtypes['AVProductStatesIdentifier'])
    df['AVProductsInstalled'] = df['AVProductsInstalled'].apply(lambda x : AVProductsInstalled_encoding(x) ).astype(dtypes['DefaultBrowsersIdentifier'])
    df['AVProductsEnabled'] = df['AVProductsEnabled'].apply(lambda x : AVProductsEnabled_encoding(x) ).astype(dtypes['AVProductsEnabled'])
    df['CountryIdentifier'] = df['CountryIdentifier'].apply(lambda x : CountryIdentifier_encoding(x) ).astype(dtypes['CountryIdentifier'])
    df['CityIdentifier'] = df['CityIdentifier'].apply(lambda x : CityIdentifier_encoding(x) ).astype(dtypes['CityIdentifier'])
    df['OrganizationIdentifier'] = df['OrganizationIdentifier'].apply(lambda x : OrganizationIdentifier_encoding(x) ).astype(dtypes['OrganizationIdentifier'])
    df['GeoNameIdentifier'] = df['GeoNameIdentifier'].apply(lambda x : GeoNameIdentifier_encoding(x) ).astype(dtypes['GeoNameIdentifier'])
    df['LocaleEnglishNameIdentifier'] = df['LocaleEnglishNameIdentifier'].apply(lambda x : LocaleEnglishNameIdentifier_encoding(x) ).astype(dtypes['LocaleEnglishNameIdentifier'])
    df['Platform'] = df['Platform'].apply(lambda x : Platform_encoding(x) ).astype(dtypes['Platform'])
    df['Processor'] = df['Processor'].apply(lambda x : Processor_encoding(x) ).astype(dtypes['Processor'])
    df['OsVer'] = df['OsVer'].apply(lambda x : OsVer_encoding(x) ).astype(dtypes['OsVer'])
    df['OsBuild'] = df['OsBuild'].apply(lambda x : OsBuild_encoding(x) ).astype(dtypes['OsBuild'])
    df['OsSuite'] = df['OsSuite'].apply(lambda x : OsSuite_encoding(x) ).astype(dtypes['OsSuite'])
    df['OsPlatformSubRelease'] = df['OsPlatformSubRelease'].apply(lambda x : OsPlatformSubRelease_encoding(x) ).astype(dtypes['OsPlatformSubRelease'])
    df['OsBuildLab'] = df['OsBuildLab'].apply(lambda x : OsBuildLab_encoding(x) ).astype(dtypes['OsBuildLab'])
    df['SkuEdition'] = df['SkuEdition'].apply(lambda x : SkuEdition_encoding(x) ).astype(dtypes['SkuEdition'])
    df['IeVerIdentifier'] = df['IeVerIdentifier'].apply(lambda x : IeVerIdentifier_encoding(x) ).astype(dtypes['IeVerIdentifier'])
    df['SmartScreen'] = df['SmartScreen'].apply(lambda x : SmartScreen_encoding(x) ).astype(dtypes['SmartScreen'])
    df['UacLuaenable'] = df['UacLuaenable'].apply(lambda x : UacLuaenable_encoding(x) ).astype(dtypes['UacLuaenable'])
    df['Census_MDC2FormFactor'] = df['Census_MDC2FormFactor'].apply(lambda x : Census_MDC2FormFactor_encoding(x) ).astype(dtypes['Census_MDC2FormFactor'])
    df['Census_DeviceFamily'] = df['Census_DeviceFamily'].apply(lambda x : Census_DeviceFamily_encoding(x) ).astype(dtypes['Census_DeviceFamily'])
    df['Census_OEMNameIdentifier'] = df['Census_OEMNameIdentifier'].apply(lambda x : Census_OEMNameIdentifier_encoding(x) ).astype(dtypes['Census_OEMNameIdentifier'])
    df['Census_OEMModelIdentifier'] = df['Census_OEMModelIdentifier'].apply(lambda x : Census_OEMModelIdentifier_encoding(x) ).astype(dtypes['Census_OEMModelIdentifier'])
    df['Census_ProcessorCoreCount'] = df['Census_ProcessorCoreCount'].apply(lambda x : Census_ProcessorCoreCount_encoding(x) ).astype(dtypes['Census_ProcessorCoreCount'])  #check once
    df['Census_ProcessorManufacturerIdentifier'] = df['Census_ProcessorManufacturerIdentifier'].apply(lambda x : Census_ProcessorManufacturerIdentifier_encoding(x) ).astype(dtypes['Census_ProcessorManufacturerIdentifier'])  #check once
    df['Census_ProcessorModelIdentifier'] = df['Census_ProcessorModelIdentifier'].apply(lambda x : Census_ProcessorModelIdentifier_encoding(x) ).astype(dtypes['Census_ProcessorModelIdentifier'])  #check once
    df['Census_ProcessorClass'] = df['Census_ProcessorClass'].apply(lambda x : Census_ProcessorClass_encoding(x) ).astype(dtypes['Census_ProcessorClass'])  
    df['Census_PrimaryDiskTypeName'] = df['Census_PrimaryDiskTypeName'].apply(lambda x : Census_PrimaryDiskTypeName_encoding(x) ).astype(dtypes['Census_PrimaryDiskTypeName']) 

    scaler = MinMaxScaler(feature_range=[0,1])
    df['Census_PrimaryDiskTotalCapacity'] = scaler.fit_transform(df['Census_PrimaryDiskTotalCapacity'].values.reshape(-1,1)).astype(dtypes['Census_PrimaryDiskTotalCapacity'])
    scaler = MinMaxScaler(feature_range=[0,1])
    df['Census_SystemVolumeTotalCapacity'] = scaler.fit_transform(df['Census_SystemVolumeTotalCapacity'].values.reshape(-1,1)).astype(dtypes['Census_SystemVolumeTotalCapacity'])

    df['Census_TotalPhysicalRAM'] = df['Census_TotalPhysicalRAM'].apply(lambda x : Census_TotalPhysicalRAM_encoding(x) ).astype(dtypes['Census_TotalPhysicalRAM'])  
    df['Census_ChassisTypeName'] = df['Census_ChassisTypeName'].apply(lambda x : Census_ChassisTypeName_encoding(x) ).astype(dtypes['Census_ChassisTypeName'])
    df['Census_InternalPrimaryDiagonalDisplaySizeInInches'] = df['Census_InternalPrimaryDiagonalDisplaySizeInInches'].apply(lambda x : Census_InternalPrimaryDiagonalDisplaySizeInInches_encoding(x) ).astype(dtypes['Census_InternalPrimaryDiagonalDisplaySizeInInches']) 
    df['Census_InternalPrimaryDisplayResolutionHorizontal'] = df['Census_InternalPrimaryDisplayResolutionHorizontal'].apply(lambda x : Census_InternalPrimaryDisplayResolutionHorizontal_encoding(x) ).astype(dtypes['Census_InternalPrimaryDisplayResolutionHorizontal']) 
    df['Census_InternalPrimaryDisplayResolutionVertical'] = df['Census_InternalPrimaryDisplayResolutionVertical'].apply(lambda x : Census_InternalPrimaryDisplayResolutionVertical_encoding(x) ).astype(dtypes['Census_InternalPrimaryDisplayResolutionVertical'])  
    df['Census_PowerPlatformRoleName'] = df['Census_PowerPlatformRoleName'].apply(lambda x : Census_PowerPlatformRoleName_encoding(x) ).astype(dtypes['Census_PowerPlatformRoleName']) 
    df['Census_InternalBatteryType'] = df['Census_InternalBatteryType'].apply(lambda x : Census_InternalBatteryType_encoding(x) ).astype(dtypes['Census_InternalBatteryType']) 
    df['Census_InternalBatteryNumberOfCharges'] = df['Census_InternalBatteryNumberOfCharges'].apply(lambda x : Census_InternalBatteryNumberOfCharges_encoding(x) ).astype(dtypes['Census_InternalBatteryNumberOfCharges']) 
    df['Census_OSVersion'] = df['Census_OSVersion'].apply(lambda x : Census_OSVersion_encoding(x) ).astype(dtypes['Census_OSVersion']) 
    df['Census_OSBranch'] = df['Census_OSBranch'].apply(lambda x : Census_OSBranch_encoding(x) ).astype(dtypes['Census_OSBranch']) 
    df['Census_OSBuildNumber'] = df['Census_OSBuildNumber'].apply(lambda x : Census_OSBuildNumber_encoding(x) ).astype(dtypes['Census_OSBuildNumber']) 

    scaler = MinMaxScaler(feature_range=[0,1])
    df['Census_OSBuildRevision'] = scaler.fit_transform(df['Census_OSBuildRevision'].values.reshape(-1,1)).astype(dtypes['Census_OSBuildRevision'])

    df['Census_OSEdition'] = df['Census_OSEdition'].apply(lambda x : Census_OSEdition_encoding(x) ).astype(dtypes['Census_OSEdition']) 
    df['Census_OSSkuName'] = df['Census_OSSkuName'].apply(lambda x : Census_OSSkuName_encoding(x) ).astype(dtypes['Census_OSSkuName']) 
    df['Census_OSInstallLanguageIdentifier'] = df['Census_OSInstallLanguageIdentifier'].apply(lambda x : Census_OSInstallLanguageIdentifier_encoding(x) ).astype(dtypes['Census_OSInstallLanguageIdentifier']) 
    df['Census_OSUILocaleIdentifier'] = df['Census_OSUILocaleIdentifier'].apply(lambda x : Census_OSUILocaleIdentifier_encoding(x) ).astype(dtypes['Census_OSUILocaleIdentifier']) 
    df['Census_ActivationChannel'] = df['Census_ActivationChannel'].apply(lambda x : Census_ActivationChannel_encoding(x) ).astype(dtypes['Census_ActivationChannel']) 
    df['Census_FirmwareManufacturerIdentifier'] = df['Census_FirmwareManufacturerIdentifier'].apply(lambda x : Census_FirmwareManufacturerIdentifier_encoding(x) ).astype(dtypes['Census_FirmwareManufacturerIdentifier'])

    scaler = MinMaxScaler(feature_range=[0,1])
    df['Census_FirmwareVersionIdentifier'] = scaler.fit_transform(df['Census_FirmwareVersionIdentifier'].values.reshape(-1,1)).astype(dtypes['Census_FirmwareVersionIdentifier'])

In [ ]:
train_df.head()

In [ ]:
gc.collect()

In [ ]:
del train_df['MachineIdentifier']
test_ids = test_df['MachineIdentifier']
del test_df['MachineIdentifier']
y = train_df['HasDetections']
del train_df['HasDetections']
train_df.shape, test_df.shape

In [ ]:
param = {
        'num_leaves': 10,
        'max_bin': 63,
        'min_data_in_leaf': 11,
        'learning_rate': 0.02,
        'min_sum_hessian_in_leaf': 0.00245,
        'bagging_fraction': 1.0, 
        'bagging_freq': 5, 
        'feature_fraction': 0.05,
        'lambda_l1': 4.972,
        'lambda_l2': 2.276,
        'min_gain_to_split': 0.65,
        'max_depth': 14,
        'save_binary': True,
        'seed': 1337,
        'feature_fraction_seed': 1337,
        'bagging_seed': 1337,
        'drop_seed': 1337,
        'data_random_seed': 1337,
        'objective': 'binary',
        'boosting_type': 'gbdt',
        'verbose': 1,
        'metric': 'auc',
        'is_unbalance': True,
        'boost_from_average': False,
    }

In [ ]:
nfold =5
k =StratifiedKFold(n_splits=nfold, shuffle=True, random_state=49)

oof = np.zeros(len(y))
predictions = np.zeros(len(test_df))

i =1

for train_idx, val_idx in k.split(train_df, y.values):
    print("\n fold {}".format(i))
    
    light_train = lightgbm.Dataset(train_df.iloc[train_idx].values,
                                  label = y.iloc[train_idx].values,
                                  free_raw_data = False)
    light_val = lightgbm.Dataset(train_df.iloc[val_idx].values,
                                label = y.iloc[val_idx].values,
                                free_raw_data = False)
    
    clf = lightgbm.train(param, light_train, 5000, valid_sets=[light_val], verbose_eval=50, early_stopping_rounds=50)
    
    oof[val_idx] = clf.predict(train_df.iloc[val_idx].values, num_iteration = clf.best_iteration)
    
    predictions += clf.predict(test_df.values, num_iteration= clf.best_iteration) /nfold
    
    i+=1

In [ ]:
sub_df = pd.DataFrame()
sub_df['MachineIdentifier'] = test_ids
sub_df['HasDetections'] = predictions
sub_df.to_csv("subLGB.csv", index=False)